In [1]:
# Getting started with NLP for absolute beginners
# https://www.kaggle.com/code/marktomm/getting-started-with-nlp-for-absolute-beginners/edit

In [2]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
creds = ''
# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

path = Path('us-patent-phrase-to-phrase-matching')

if not iskaggle and not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

if iskaggle:
    path = Path('../input/us-patent-phrase-to-phrase-matching')
    ! pip install -q datasets

In [3]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [4]:
import pandas as pd
df = pd.read_csv(path/'train.csv')
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [5]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


In [6]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + \
df.target + '; ANC1: ' + df.anchor

In [7]:
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [8]:
from datasets import Dataset,DatasetDict
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [9]:
model_nm = 'microsoft/deberta-v3-small'

In [10]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

/home/mark/miniconda3/envs/py10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/mark/miniconda3/envs/py10/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sur

In [11]:
tokz.tokenize('Hell\'o world!;')

['▁Hell', "'", 'o', '▁world', '!', ';']

In [12]:
def tok_func(x): return tokz(x['input'])

In [13]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [14]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [15]:
tokz.vocab['▁of']

265

In [16]:
# rename score to labels
tok_ds = tok_ds.rename_columns({'score': 'labels'})

In [17]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,hybrid bearing,inorganic photoconductor drum,G02
freq,1,2,1,3


In [18]:
eval_df.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [19]:
import numpy as np, matplotlib.pyplot as plt

In [20]:
from numpy.random import normal,seed,uniform
np.random.seed(42)

In [21]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [22]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [23]:
eval_df.head(), eval_ds

(                 id              anchor                         target  \
 0  4112d61851461f60            opc drum  inorganic photoconductor drum   
 1  09e418c93a776564     adjust gas flow              altering gas flow   
 2  36baf228038e314b      lower trunnion                 lower locating   
 3  1f37ead645e7f0c8       cap component                  upper portion   
 4  71a5b6ad068d531f  neural stimulation      artificial neural network   
 
   context                                              input  
 0     G02  TEXT1: G02; TEXT2: inorganic photoconductor dr...  
 1     F23  TEXT1: F23; TEXT2: altering gas flow; ANC1: ad...  
 2     B60  TEXT1: B60; TEXT2: lower locating; ANC1: lower...  
 3     D06  TEXT1: D06; TEXT2: upper portion; ANC1: cap co...  
 4     H04  TEXT1: H04; TEXT2: artificial neural network; ...  ,
 Dataset({
     features: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 36
 }))

In [24]:
np.set_printoptions(precision=2, suppress=True)
def corr(x,y): return np.corrcoef(x,y)[0][1]
def show_corr(df, a, b):
    x,y = df[a],df[b]
    plt.scatter(x,y, alpha=0.5, s=4)
    plt.title(f'{a} vs {b}; r: {corr(x, y):.2f}')
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [25]:
from transformers import TrainingArguments,Trainer
bs = 32
epochs = 4
lr = 8e-5
args = TrainingArguments(
    'outputs', 
    learning_rate=lr, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine', 
    fp16=True,
    evaluation_strategy="epoch", 
    per_device_train_batch_size=bs, 
    per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, 
    weight_decay=0.01, 
    report_to='none'
)

/home/mark/miniconda3/envs/py10/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [26]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_nm, num_labels = 1
)
trainer = Trainer(model, args, train_dataset = dds['train'],
                  eval_dataset = dds['test'], tokenizer=tokz, 
                  compute_metrics = corr_d)

/home/mark/miniconda3/envs/py10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'pooler.dense.bias', 'classifier.weight', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/mark/miniconda3/envs/py10/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoade

In [27]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Pearson
1,0.045200,0.027390,0.792429
2,0.021800,0.023863,0.820665
3,0.013200,0.021010,0.833974
4,0.010100,0.021558,0.836292


TrainOutput(global_step=3420, training_loss=0.02111920794548347, metrics={'train_runtime': 5085.8179, 'train_samples_per_second': 21.514, 'train_steps_per_second': 0.672, 'total_flos': 662735834465040.0, 'train_loss': 0.02111920794548347, 'epoch': 4.0})

In [29]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.48],
       [ 0.72],
       [ 0.57],
       [ 0.28],
       [-0.03],
       [ 0.51],
       [ 0.5 ],
       [-0.04],
       [ 0.26],
       [ 1.06],
       [ 0.25],
       [ 0.25],
       [ 0.73],
       [ 0.96],
       [ 0.77],
       [ 0.31],
       [ 0.27],
       [-0.03],
       [ 0.6 ],
       [ 0.41],
       [ 0.44],
       [ 0.25],
       [ 0.14],
       [ 0.25],
       [ 0.53],
       [-0.01],
       [-0.03],
       [-0.03],
       [-0.02],
       [ 0.59],
       [ 0.38],
       [ 0.06],
       [ 0.74],
       [ 0.51],
       [ 0.39],
       [ 0.26]])

In [30]:
preds = np.clip(preds, 0, 1)
preds

array([[0.48],
       [0.72],
       [0.57],
       [0.28],
       [0.  ],
       [0.51],
       [0.5 ],
       [0.  ],
       [0.26],
       [1.  ],
       [0.25],
       [0.25],
       [0.73],
       [0.96],
       [0.77],
       [0.31],
       [0.27],
       [0.  ],
       [0.6 ],
       [0.41],
       [0.44],
       [0.25],
       [0.14],
       [0.25],
       [0.53],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.59],
       [0.38],
       [0.06],
       [0.74],
       [0.51],
       [0.39],
       [0.26]])

In [31]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

855

In [34]:
trainer.save_model('hfmodel')

In [35]:
model2 = AutoModelForSequenceClassification.from_pretrained('hfmodel/')

In [37]:
trainer2 = Trainer(model2, args, tokenizer=tokz, 
                  compute_metrics = corr_d)

/home/mark/miniconda3/envs/py10/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [38]:
trainer2.predict(eval_ds).predictions.astype(float)

array([[ 0.48],
       [ 0.72],
       [ 0.57],
       [ 0.28],
       [-0.03],
       [ 0.51],
       [ 0.5 ],
       [-0.04],
       [ 0.26],
       [ 1.06],
       [ 0.25],
       [ 0.25],
       [ 0.73],
       [ 0.96],
       [ 0.77],
       [ 0.31],
       [ 0.27],
       [-0.03],
       [ 0.6 ],
       [ 0.41],
       [ 0.44],
       [ 0.25],
       [ 0.14],
       [ 0.25],
       [ 0.53],
       [-0.01],
       [-0.03],
       [-0.03],
       [-0.02],
       [ 0.59],
       [ 0.38],
       [ 0.06],
       [ 0.74],
       [ 0.51],
       [ 0.39],
       [ 0.26]])